In [ ]:
import kagglehub
kagglehub.login()

Kaggle credentials set.
Kaggle credentials successfully validated.


In [ ]:
esa_adb_challenge_path = kagglehub.competition_download('esa-adb-challenge')
root = esa_adb_challenge_path

print("⇲ Dataset Loaded...")

100%|██████████| 282M/282M [00:01<00:00, 192MB/s]

Extracting files...


⇲ Dataset Loaded...


In [ ]:
import numpy as np
import pandas as pd

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
root = esa_adb_challenge_path
df = pd.read_parquet(f"{root}/train.parquet")
df_test = pd.read_parquet(f"{root}/test.parquet")

df_test.head()

,id,channel_1,channel_10,channel_11,channel_12,channel_13,channel_14,channel_15,channel_16,channel_17,...,telecommand_350,telecommand_351,telecommand_352,telecommand_353,telecommand_354,telecommand_36,telecommand_376,telecommand_38,telecommand_39,telecommand_40
0,14728321,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0
1,14728322,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0
2,14728323,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0
3,14728324,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0
4,14728325,0.13791,0.0,0.0,0.218915,0.270384,0.63107,0.314563,0.786995,0.265045,...,0,0,0,0,0.0,0.0,0,0.0,0.0,0.0


In [ ]:
!pip install pandas numpy torch pyro-ppl scikit-learn matplotlib pyarrow tabulate pgmpy -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 756.0/756.0 kB 14.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 62.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 82.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 232.9/232.9 kB 18.0 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/env python3
# -*- coding: utf-8 -*-

import os
import time
import json
import logging
import pickle

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
from tabulate import tabulate
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_score, recall_score, fbeta_score
from pgmpy.estimators import PC
from pgmpy.models import DynamicBayesianNetwork
from pgmpy.estimators import MaximumLikelihoodEstimator

# ──────────────────────────────────────────────────────────────────────────────
# CONFIGURATION
# ──────────────────────────────────────────────────────────────────────────────
CONFIG = {
    "paths": {
        "train":         f"{root}/train.parquet",
        "test":          f"{root}/test.parquet",
        "artifacts_dir": "artifacts/",
        "preproc_train": "artifacts/preprocessed_train.parquet",
        "preproc_test":  "artifacts/preprocessed_test.parquet",
        "parents_map":   "artifacts/parents_map.json",
        "dbn_model":     "artifacts/dbn_model.pkl",
        "train_scores":  "artifacts/train_scores.parquet",
        "test_scores":   "artifacts/test_scores.parquet",
        "predictions":   "artifacts/predictions.parquet",
        "plots_dir":     "artifacts/plots/",
        "log_file":      "artifacts/pipeline.log"
    },
    "preprocessing": {
        "scale": True,
        "scaler": "standard",   # "standard" or "minmax"
        "windowing": {
            "use":     True,
            "size":    100_000,
            "overlap": 10_000
        }
    },
    "structure_learning": {
        "method":          "PC",           # only PC supported here
        "ci_test":         "chi_square",   # or "hellinger"
        "significance":    0.01,
        "max_cond_vars":   2
    },
    "dbn": {
        "intra_slice_edges":  True,  # use discovered intra-slice structure
        "temporal_self_edges": True   # add (X_t-1)->(X_t) for each variable
    },
    "detection": {
        "threshold_quantile": 0.999
    },
    "evaluation": {
        "compute_metrics": True
    },
    "logging": {
        "level": logging.INFO
    }
}

# ──────────────────────────────────────────────────────────────────────────────
# SETUP
# ──────────────────────────────────────────────────────────────────────────────
os.makedirs(CONFIG["paths"]["artifacts_dir"], exist_ok=True)
os.makedirs(CONFIG["paths"]["plots_dir"], exist_ok=True)

# Logging to console + file
logger = logging.getLogger("ESA-DBN-Pipeline")
logger.setLevel(CONFIG["logging"]["level"])
fmt_console = logging.Formatter("%(asctime)s | %(levelname)s | %(message)s", "%H:%M:%S")
fmt_file    = logging.Formatter("%(asctime)s %(levelname)s %(message)s", "%Y-%m-%d %H:%M:%S")
ch = logging.StreamHandler(); ch.setFormatter(fmt_console); logger.addHandler(ch)
fh = logging.FileHandler(CONFIG["paths"]["log_file"], mode="w"); fh.setFormatter(fmt_file); logger.addHandler(fh)

# ──────────────────────────────────────────────────────────────────────────────
# UTILITY FUNCTIONS
# ──────────────────────────────────────────────────────────────────────────────

def timeit(func):
    """Decorator to log a function's runtime."""
    def wrapper(*args, **kwargs):
        start = time.time()
        res = func(*args, **kwargs)
        elapsed = time.time() - start
        logger.info(f"Completed `{func.__name__}` in {elapsed:.1f}s")
        return res
    return wrapper

def tabulate_metrics(metrics: dict, title: str):
    """Print a small table of metrics."""
    table = [[k, f"{v:.4f}"] for k,v in metrics.items()]
    print(f"\n[{title}]")
    print(tabulate(table, headers=["Metric", "Value"], tablefmt="github"), "\n")

# ──────────────────────────────────────────────────────────────────────────────
# 1) DATA LOADING
# ──────────────────────────────────────────────────────────────────────────────
@timeit
def load_data():
    train = pd.read_parquet(CONFIG["paths"]["train"])
    test  = pd.read_parquet(CONFIG["paths"]["test"])
    logger.info(f"Train shape: {train.shape}, Test shape: {test.shape}")
    return train, test

# ──────────────────────────────────────────────────────────────────────────────
# 2) PREPROCESSING & WINDOWING
# ──────────────────────────────────────────────────────────────────────────────
@timeit
def preprocess_and_window(train: pd.DataFrame, test: pd.DataFrame):
    print("PROCESSING DATA")
    feats = [c for c in train.columns if c not in ("id","is_anomaly")]
    # Scaling
    if CONFIG["preprocessing"]["scale"]:
        logger.info(f"Fitting scaler on {len(feats)} features")
        if CONFIG["preprocessing"]["scaler"]=="standard":
            scaler = StandardScaler()
        else:
            from sklearn.preprocessing import MinMaxScaler
            scaler = MinMaxScaler()
        scaler.fit(train[train.is_anomaly==0][feats])
        # apply with tqdm
        for df,label in [(train,"train"),(test,"test")]:
            for col in tqdm(feats, desc=f"Scaling {label}", unit="feat"):
                df[col] = (df[col] - scaler.mean_[feats.index(col)]) / scaler.scale_[feats.index(col)]
    # Save preprocessed
    train.to_parquet(CONFIG["paths"]["preproc_train"], index=False)
    test .to_parquet(CONFIG["paths"]["preproc_test"],  index=False)
    logger.info("Saved preprocessed data")
    # Windowing (train only, for structure/parameter learning)
    win_cfg = CONFIG["preprocessing"]["windowing"]
    windows = []
    if win_cfg["use"]:
        w, o = win_cfg["size"], win_cfg["overlap"]
        logger.info(f"Windowing train into size={w}, overlap={o}")
        idx = 0; n = len(train)
        while idx < n:
            end = min(idx+w, n)
            windows.append(train.iloc[idx:end].copy())
            idx += w - o
        logger.info(f"Created {len(windows)} windows")
    else:
        windows = [train]
    return train, test, feats, windows

# ──────────────────────────────────────────────────────────────────────────────
# 3) CAUSAL STRUCTURE LEARNING (PC)
# ──────────────────────────────────────────────────────────────────────────────
@timeit
def learn_causal_structure(windows, feats):
    # Concatenate first few windows up to ~1e6 points to limit cost
    sample = pd.concat(windows)[:1_000_000]
    data = sample[feats]
    logger.info(f"Running PC on sample of {len(data)} rows")
    pc = PC(data)
    model = pc.estimate(
        significance_level=CONFIG["structure_learning"]["significance"],
        variant="orig",
        ci_test=CONFIG["structure_learning"]["ci_test"],
        max_cond_vars=CONFIG["structure_learning"]["max_cond_vars"]
    )
    edges = list(model.edges())
    logger.info(f"PC learned {len(edges)} edges")
    # Save as parents_map
    parents = {v: [] for v in feats}
    for u,v in edges:
        if u in feats and v in feats:
            parents[v].append(u)
    with open(CONFIG["paths"]["parents_map"], "w") as f:
        json.dump(parents, f, indent=2)
    logger.info("Saved parents_map.json")
    return parents

# ──────────────────────────────────────────────────────────────────────────────
# 4) BUILD & TRAIN DYNAMIC BAYESIAN NETWORK
# ──────────────────────────────────────────────────────────────────────────────
@timeit
def build_and_train_dbn(windows, feats, parents):
    dbn = DynamicBayesianNetwork()
    # Add intra-slice edges
    if CONFIG["dbn"]["intra_slice_edges"]:
        for u in parents:
            for v in parents[u]:
                dbn.add_edge((u, 0), (u, 1))  # self temporal
                dbn.add_edge((v, 0), (u, 0))
    # Add temporal self-edges
    if CONFIG["dbn"]["temporal_self_edges"]:
        for u in feats:
            dbn.add_edge((u, 0), (u, 1))
    # Convert windows into training dataset for DBN parameter learning
    # Need 2-slice table: for each window, pairs of t-1,t
    dfs = []
    for w in windows:
        df = w[feats + ["id"]].copy()
        df.columns = [f"{c}_0" for c in feats] + ["id_0"]
        df1 = df.copy()
        df1.columns = [f"{c}_1" for c in feats] + ["id_1"]
        paired = pd.concat([df.reset_index(drop=True), df1.shift(-1).reset_index(drop=True)], axis=1).dropna()
        dfs.append(paired)
    train2 = pd.concat(dfs).drop(columns=["id_1"])
    # Fit CPDs
    dbn.fit(train2, estimator=MaximumLikelihoodEstimator)
    # Save model
    with open(CONFIG["paths"]["dbn_model"], "wb") as f:
        pickle.dump(dbn, f)
    logger.info("Saved DBN model pickle")
    return dbn

# ──────────────────────────────────────────────────────────────────────────────
# 5) COMPUTE ANOMALY SCORES
# ──────────────────────────────────────────────────────────────────────────────
@timeit
def compute_scores(dbn, df, feats, name):
    logger.info(f"Scoring {name} ({len(df)} points)")
    scores = np.zeros(len(df))
    data = df.sort_values("id")
    # Compute log likelihood of each row given previous row
    # We must iterate row by row (pgmpy limitation), but batch windows:
    prev = data.shift(1).dropna()
    curr = data.loc[prev.index]
    # Prepare DataFrame with MultiIndex columns matching DBN nodes
    joint = pd.concat([
        prev[feats].rename(columns=lambda c: (c,0)),
        curr[feats].rename(columns=lambda c: (c,1))
    ], axis=1)
    logp = dbn.log_probability(joint)
    scores[1:] = -logp  # first is zero
    # Save
    out = pd.DataFrame({"id": df["id"], "score": scores})
    out.to_parquet(CONFIG["paths"][f"{name}_scores"], index=False)
    logger.info(f"Saved {name}_scores.parquet")
    return scores

# ──────────────────────────────────────────────────────────────────────────────
# 6) THRESHOLD & PREDICT
# ──────────────────────────────────────────────────────────────────────────────
@timeit
def threshold_and_predict(train_scores, test_scores):
    q = CONFIG["detection"]["threshold_quantile"]
    thr = np.quantile(train_scores, q)
    preds = (test_scores >= thr).astype(int)
    logger.info(f"Threshold at {q*100:.2f}% quantile = {thr:.3f}; flagged {preds.sum()} anomalies")
    # Save predictions
    out = pd.DataFrame({"id": test_df["id"], "is_anomaly": preds})
    out.to_parquet(CONFIG["paths"]["predictions"], index=False)
    logger.info("Saved predictions.parquet")
    return preds, thr

# ──────────────────────────────────────────────────────────────────────────────
# 7) EVALUATION
# ──────────────────────────────────────────────────────────────────────────────
def evaluate(y_true, y_pred):
    # Sample-level
    p = precision_score(y_true, y_pred)
    r = recall_score(y_true, y_pred)
    f05 = fbeta_score(y_true, y_pred, beta=0.5)
    # Event-level corrected F0.5
    def events(z):
        ev=[]; on=False
        for i,v in enumerate(z):
            if v and not on: on, s = True, i
            if not v and on: on=False; ev.append((s,i-1))
        if on: ev.append((s,len(z)-1))
        return ev
    te, pe = events(y_true), events(y_pred)
    TPe = sum(1 for ps,pe_ in pe if any(ps<=te_ and pe_>=ts for ts,te_ in te))
    FPe = len(pe) - TPe
    FPt = np.sum((y_pred==1)&(y_true==0))
    Nt  = np.sum(y_true==0)
    Prec_corr = TPe / (TPe + FPe*(1 - FPt/Nt)) if (TPe+FPe)>0 else 0
    Rec_e     = TPe / len(te) if te else 0
    f05_e = (1+0.5**2)*Prec_corr*Rec_e / (0.5**2*Prec_corr + Rec_e) if (Prec_corr+Rec_e)>0 else 0
    return {
        "sample_p": p, "sample_r": r, "sample_F0.5": f05,
        "event_p_corr": Prec_corr, "event_r": Rec_e, "event_F0.5": f05_e
    }

# ──────────────────────────────────────────────────────────────────────────────
# MAIN EXECUTION
# ──────────────────────────────────────────────────────────────────────────────
if __name__ == "__main__":
    logger.info("=== ESA-ADB Anomaly Detection Pipeline START ===")
    # 1. Load
    train, test = load_data()
    # 2. Preprocess/Window
    train_p, test_p, feats, windows = preprocess_and_window(train, test)
    # 3. Structure
    parents = learn_causal_structure(windows, feats)
    # 4. DBN build & train
    dbn = build_and_train_dbn(windows, feats, parents)
    # 5. Score
    train_scores = compute_scores(dbn, train_p, feats, "train")
    test_scores  = compute_scores(dbn, test_p, feats, "test")
    # 6. Predict
    preds, thr  = threshold_and_predict(train_scores, test_scores)
    # 7. Evaluate
    if CONFIG["evaluation"]["compute_metrics"]:
        metrics = evaluate(train_p["is_anomaly"].values, train_scores>=thr)
        tabulate_metrics(metrics, "TRAIN METRICS")
        metrics_test = evaluate(test_p["is_anomaly"].values, preds)
        tabulate_metrics(metrics_test, "TEST METRICS")
    logger.info("=== Pipeline COMPLETE ===")


01:04:02 | INFO | === ESA-ADB Anomaly Detection Pipeline START ===
INFO:ESA-DBN-Pipeline:=== ESA-ADB Anomaly Detection Pipeline START ===
01:04:03 | INFO | Train shape: (14728321, 89), Test shape: (521280, 88)
INFO:ESA-DBN-Pipeline:Train shape: (14728321, 89), Test shape: (521280, 88)
01:04:03 | INFO | Completed `load_data` in 0.9s
INFO:ESA-DBN-Pipeline:Completed `load_data` in 0.9s
01:04:03 | INFO | Fitting scaler on 87 features
INFO:ESA-DBN-Pipeline:Fitting scaler on 87 features


PROCESSING DATA


Scaling test: 100%|██████████| 87/87 [00:00<00:00, 450.77feat/s]
01:04:51 | INFO | Saved preprocessed data
INFO:ESA-DBN-Pipeline:Saved preprocessed data
01:04:51 | INFO | Windowing train into size=100000, overlap=10000
INFO:ESA-DBN-Pipeline:Windowing train into size=100000, overlap=10000
01:04:56 | INFO | Created 164 windows
INFO:ESA-DBN-Pipeline:Created 164 windows
01:04:56 | INFO | Completed `preprocess_and_window` in 53.4s
INFO:ESA-DBN-Pipeline:Completed `preprocess_and_window` in 53.4s
01:04:59 | INFO | Running PC on sample of 1000000 rows
INFO:ESA-DBN-Pipeline:Running PC on sample of 1000000 rows


  0%|          | 0/5 [00:00<?, ?it/s]

In [ ]:
from google.colab import drive
drive.mount('/content/drive')